# 실습 2. CRM 데이터 고객 세분화
1. 데이터
: Online Retail.csv
2. 예측
: Retail 구매에 따른 고객 세분화
(Segmentaiton
)
3. 특징
: 군집(Clustering) 분석
4. 목표
:
• 군집 알고리즘 사용 • 'TotalSales', 'OrderCount', 'AvgOrderValue’ 전처리 • 위의 Feature로 고객 세분화(Clustering

## Hint
1. Data load  
1. Data Clean-up  
- 0 이상인 것만, drop null  
- InvoiceData 20111201 이상만  
- total sales 구하기
- customerid로 group by, invoiceNo는 nunique로
- 고객마다 몇번 구매한지 구함
- rank 메서드로 매출 순위 내림차순
3. Customer Segmentation via K means Clustering
- Clustering label 넣기
- 클러스터링 시각화 해보기 scatter
- K값에 따라 
